In [6]:
# Cell 1: Set up the path
import sys
import os
from dotenv import load_dotenv

# Load the .env file
load_dotenv()
# Get the SRC_PATH from the environment variable
src_path = os.getenv('SRC_PATH')

print(f"SRC_PATH: {src_path}")

if src_path:
    # Add the src path to sys.path
    sys.path.append(os.path.abspath(src_path))

SRC_PATH: D:\CODE\de_projects\finance_hub\src


In [7]:
from pipeline.commons.constants import VnstockDataSources
from pipeline.commons.connectors.vnstock_lib import VnstockLibConnector

## Connector

In [8]:
connector = VnstockLibConnector(init_symbol='MBB')
connector.connect()

2024-10-25 14:14:55,363 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


Connecting to VnstockLib: Symbol: MBB, Source: VCI
Connected to VnstockLib.


In [9]:
connector.list_all_symbols()

,ticker,organ_name
0,A32,Công ty Cổ phần 32
1,AAA,Công ty Cổ phần Nhựa An Phát Xanh
2,AAM,Công ty Cổ Phần Thủy Sản MeKong
3,AAS,Công ty Cổ phần Chứng khoán SmartInvest
4,AAT,Công ty Cổ phần Tập đoàn Tiên Sơn Thanh Hóa
...,...,...
1593,XPH,Công ty Cổ phần Xà phòng Hà Nội
1594,YBC,Công ty Cổ phần Xi măng và Khoáng sản Yên Bái
1595,YBM,Công ty Cổ phần Khoáng sản Công nghiệp Yên Bái
1596,YEG,Công ty Cổ phần Tập đoàn Yeah1


## Extractor

## Transformer

## Loader